In [1]:
!pip install mlflow

     |████████████████████████████████| 10.4MB 2.6MB/s 
     |████████████████████████████████| 153kB 42.6MB/s 
     |████████████████████████████████| 1.1MB 50.3MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 6.0MB 42.2MB/s 
     |████████████████████████████████| 81kB 10.0MB/s 
     |████████████████████████████████| 92kB 11.1MB/s 
     |████████████████████████████████| 51kB 6.9MB/s 
     |████████████████████████████████| 460kB 46.0MB/s 
     |████████████████████████████████| 204kB 44.1MB/s 
     |████████████████████████████████| 81kB 10.6MB/s 
     |████████████████████████████████| 71kB 9.8MB/s 
  Created wheel for alembic: filename=alembic-1.4.2-cp36-none-any.whl size=159543 sha256=eaad9b682b2c776daae29bda33e9fa7f063ed55c30de0481b461aa5df1acf2f5
  Stored in directory: /root/.cache/pip/wheels/1f/04/83/76023f7a4c14688c0b5c2682a96392cfdd3ee4449eaaa287ef
Succ

In [2]:
import pandas as pd
import numpy as np
from datetime  import datetime  
from datetime import timedelta  
import lightgbm as lgb
from sklearn import preprocessing, metrics
from sklearn.model_selection import ParameterGrid
import matplotlib.pyplot as plt
import seaborn as sns
import mlflow
import mlflow.lightgbm
import seaborn as sns

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
np.random.seed(0)

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
data_folder = '/content/drive/My Drive/Kaggle_M5/00_data/'
submission = pd.read_csv(data_folder + 'sample_submission.csv')

In [0]:
params = {
        'boosting_type': 'gbdt',
        'metric': 'rmse',
        'objective':  'tweedie',
        'n_jobs': -1,
        'seed': 0,
        'learning_rate': 0.05, 
        'bagging_fraction': 0.9,
        'bagging_freq': 1, 
        'colsample_bytree': 0.75,
        'num_iterations': 5000,
        'early_stopping_round': 500,
        'max_bin': 100,
        'boost_from_average': False}

In [0]:
params = {'bagging_fraction': 0.5, 
          'bagging_freq': 10, 
          'boosting_type': 'gbdt', 
          'colsample_bytree': 0.75, 
          'early_stopping_round': 500, 
          'learning_rate': 0.05, 
          'metric': 'rmse', 
          'n_jobs': -1, 
          'num_iterations': 5000, 
          'early_stopping_round': 500,         
          'objective': 'tweedie', 
          'seed': 0}

In [0]:
evaluation_rows = [row for row in submission['id'] if 'evaluation' in row] 
evaluation = submission[submission['id'].isin(evaluation_rows)]

validation_rows = [row for row in submission['id'] if 'validation' in row] 
validation_csv = submission[submission['id'].isin(validation_rows)]


In [0]:
not_features = ['d', 'id', 'item_id', 'store_id', 'demand', 'date', 'start_date']

In [0]:
validation = pd.DataFrame(columns=submission.columns)
data_folder = '/content/drive/My Drive/Kaggle_M5/01_preprocessed_data/'

overall_rmse = 0
pred_true = pd.DataFrame(columns=['pred', 'true'])
rmse_score = []
mae_score = []
j = 0
for i in ['CA_1', 'CA_2', 'CA_3', 'CA_4', 'TX_1', 'TX_2', 'TX_3', 'WI_1', 'WI_2', 'WI_3']:
  print(i)
  X_val = pd.read_pickle(data_folder + "X_val_{}.pkl".format(i))
  X_test = pd.read_pickle(data_folder + "X_test_{}.pkl".format(i))
  X_train = pd.read_pickle(data_folder + "X_train_{}.pkl".format(i))
  y_train = X_train['demand']
  y_val = X_val['demand']
  print(X_train.shape)
  features = X_train.columns[~X_train.columns.isin(not_features) ]

  train_set = lgb.Dataset(X_train[features],y_train)
  val_set = lgb.Dataset(X_val[features], y_val)
  model = lgb.train(params, train_set, valid_sets = [train_set, val_set], verbose_eval = 100)
  
  val_pred = model.predict(X_val[features])
  val_score = metrics.mean_absolute_error(val_pred, y_val)

  temp = pd.DataFrame({'pred':val_pred, 'true':y_val})
  pred_true =pred_true.append(temp)
  print(f'Our val mae score is {val_score}')

  y_test = model.predict(X_test[features])
  X_test['demand'] = y_test
  predictions = X_test[['id', 'days_from_start', 'demand']]
  predictions = pd.pivot(predictions, index = 'id', columns = 'days_from_start', values = 'demand').reset_index()
  predictions.columns = ['id'] + ['F' + str(j + 1) for j in range(28)]
  validation = validation.append(predictions)

rmse = np.sqrt(np.mean((pred_true['pred'] - pred_true['true'])**2) )
print(rmse)
print(np.mean(np.abs(pred_true['pred'] - pred_true['true']) ))
    

In [0]:
np.sqrt(np.mean((pred_true['pred'] - pred_true['true'])**2) )

In [0]:
np.mean(np.abs(pred_true['pred'] - pred_true['true']) )

In [0]:
validation = validation.set_index('id')
validation = validation.reindex(index=validation_csv['id'])
validation = validation.reset_index()

In [0]:
validation

In [0]:
final = pd.concat([validation, evaluation])

In [0]:
submission_folder = '/content/drive/My Drive/Kaggle_M5/04_submissions/'
final.to_csv(submission_folder  + 'submission-by_store.csv', index = False)